# Advanced guide for data modeling based cogshop runs

**This tutorial covers more advanced techniques than quickstart and will not go into details covered there, such as setting up the Powerops Client**


## Preparation


NOTE: The "recommended" way of running CogShop is as follows:
1. Static model data and mappings for time-dependent data like time series represented by `ShopModel`
2. Any variations on the base configuration (like different price scenarios) represented by `ShopScenario` instances 
3. Generate a complete `ShopCase` with time series datapoints based on a `ShopScenario` instance plus `startTime` and `endTime` through the shop_trigger Cognite function.

This tutorial covers a different set-up: Triggering (Cog)Shop for a pre-generated, complete Shop "case" (as a yaml file or separate files).


In [ ]:
# Setting up `PowerOpsClient`
# You can control which setting files are loaded through the environmental variable below.
import os
from cognite.powerops._version import __version__
from cognite.powerops import PowerOpsClient

# NB! Auth has changed, this must change

os.environ["SETTINGS_FILES"] = "../../settings.toml;../../.secrets.toml"
# powerops = PowerOpsClient.from_config("../../power_ops_config.yaml")
powerops = PowerOpsClient.from_settings()

print(__version__)  # Print the version of the package

0.102.3


In [2]:
# Creating some helper functions

from pathlib import Path


def upload_file(local_file_name: str, external_id: str) -> str:
    """Upload a file to CDF and return the external id of the file"""
    file_path = (Path("example_case_files") / local_file_name).resolve()
    file = powerops.cdf.files.upload(
        path=str(file_path),
        external_id=external_id,
        name=local_file_name,
        data_set_id=powerops.datasets.write_dataset_id,
        mime_type="application/yaml",
        # Overwrite the file at the given external is if it already exists
        # This will also overwrite potentially existing metadata
        overwrite=True,
    )
    return file.external_id

## Example 1: Use multiple files for shop files
Very similar to the example in quick start, but using multiple files for `ShopCase`.

1. Upload the files that will be loaded into shop, keeping their external ids
2. Set up a `ShopCase`, using the SDK
3. Write the `ShopCase` instance to CDF, and verify that it was created using the SDK
4. Trigger a shop execution of that `ShopCase`
5. List the `ShopResult` referencing the `ShopCase` 


### Step 1: Upload a files 

In [ ]:
# NB! The case in this example is minimal and just for demo purposes.
example_case_fbu_1 = upload_file(
    local_file_name="b_example_fornebu_without_commands.yaml",
    external_id="example_case_fornebu",
)
example_case_fbu_2 = upload_file(
    local_file_name="b_example_commands.yaml", external_id="example_commands"
)

print(
    f" File reference 1: '{example_case_fbu_1}', File reference 2: '{example_case_fbu_2}'"
)

 File reference 1: 'example_case_fornebu', File reference 2: 'example_commands'


### Step 2: Preparing a ShopCase

The main difference here is that `shop_file_list` will contain more than one tuple.

Unlike the quickstart, in this example we will let `prepare_shop_case` generate all the external IDs.

We will also let the file reference double as the name of the file. 


In [4]:
import datetime


#  Files are referenced using 4-tuples with the following structure:
# (file_reference, file_name, is_ascii, labels)
shop_file_list = [
    (example_case_fbu_1, example_case_fbu_1, False, ""),
    # CogShop needs to know which file contains commands in the case of multiple files
    (example_case_fbu_2, example_case_fbu_2, False, "commands"),
]


shop_case_1 = powerops.cogshop.prepare_shop_case(
    shop_file_list=shop_file_list,
    shop_version="15.6.1.0",
    start_time=datetime.datetime(2023, 1, 19, 23),
    end_time=datetime.datetime(2023, 1, 29, 23),
    model_name="my_model",
    scenario_name="my_scenario",
)
shop_case_1

,value
space,power_ops_instances
external_id,shopcase:c2ca6f614a074a8ebbc93f18b64160f4
data_record,{'existing_version': None}
node_type,None
scenario,"{'space': 'power_ops_instances', 'external_id'..."
start_time,2023-01-19 23:00:00
end_time,2023-01-29 23:00:00
shop_files,"[{'space': 'power_ops_instances', 'external_id..."


In [5]:
# We can also look at just the shop files
for file in shop_case_1.shop_files:
    print(
        f"File name: '{file.name}', File reference: '{file.file_reference}' \t[,data modeling external id: '{file.external_id}']"
    )

File name: 'example_case_fornebu', File reference: 'example_case_fornebu' 	[,data modeling external id: 'shopfile:a5045777f9894fdba74957c40404dc2a']
File name: 'example_commands', File reference: 'example_commands' 	[,data modeling external id: 'shopfile:31229a0ace1a4c139f8d8c7be9793fb1']


### Step 3: Upload the ShopCase

Note: The upload is an `upsert` -- so if there are overlaps, the existing properties will merge  


In [6]:
powerops.cogshop.write_resources(shop_case_1)

ResourcesWriteResult(nodes=[<NodeApplyResult(space='power_ops_instances', external_id='shopcase:c2ca6f614a074a8ebbc93f18b64160f4', version=1) at 0x7f1432ab9d90>, <NodeApplyResult(space='power_ops_instances', external_id='shopfile:a5045777f9894fdba74957c40404dc2a', version=1) at 0x7f1489246c90>, <NodeApplyResult(space='power_ops_instances', external_id='shopfile:31229a0ace1a4c139f8d8c7be9793fb1', version=1) at 0x7f143337a6d0>, <NodeApplyResult(space='power_ops_instances', external_id='shopscenario:f6e3b243ff7a4413a1c5d32a0f88eac1', version=1) at 0x7f1433367b50>, <NodeApplyResult(space='power_ops_instances', external_id='shopmodel:aa8cdf7f19ef44faaf6ab3a00a7928db', version=1) at 0x7f14328e6e90>], edges=[<EdgeApplyResult(space='power_ops_instances', external_id='shopcase:c2ca6f614a074a8ebbc93f18b64160f4:shopfile:a5045777f9894fdba74957c40404dc2a', version=1) at 0x7f1433209d10>, <EdgeApplyResult(space='power_ops_instances', external_id='shopcase:c2ca6f614a074a8ebbc93f18b64160f4:shopfile:312

In [7]:
from cognite.powerops.client._generated.v1.data_classes import ShopCase

# NB! This step is not necessary, if the upsert was successful we know the case is in CDF
retrieved_shop_case: ShopCase = powerops.cogshop.retrieve_shop_case(
    shop_case_1.external_id
)
retrieved_shop_case

,value
space,power_ops_instances
external_id,shopcase:c2ca6f614a074a8ebbc93f18b64160f4
data_record,"{'version': 1, 'last_updated_time': 2024-11-11..."
node_type,None
scenario,shopscenario:f6e3b243ff7a4413a1c5d32a0f88eac1
start_time,2023-01-19 23:00:00+00:00
end_time,2023-01-29 23:00:00+00:00
shop_files,"[shopfile:a5045777f9894fdba74957c40404dc2a, sh..."


Alternately, use Graph QL to retrieve the Shop Case instance. 
Under the hood, this uses a different method to look for the case so the returned data has a slightly different format

In [8]:
graph_ql_shop_case = powerops.cogshop.retrieve_shop_case_graphql(
    shop_case_1.external_id
)
graph_ql_shop_case

,value
space,power_ops_instances
external_id,shopcase:c2ca6f614a074a8ebbc93f18b64160f4
data_record,"{'version': 0, 'last_updated_time': 2024-11-11..."
node_type,None
scenario,"{'space': 'power_ops_instances', 'external_id'..."
start_time,2023-01-19 23:00:00+00:00
end_time,2023-01-29 23:00:00+00:00
shop_files,"[{'space': 'power_ops_instances', 'external_id..."


### Step 4: Trigger the case

We will here directly use the new `trigger_shop_case` method on the `powerops.cogshop` client

In [9]:
powerops.cogshop.trigger_shop_case(shop_case_1.external_id)

### Step 5: View the ShopResult generated 

Because it is the case that a single case can be triggered multiple times, lists will be returned. 


Note that it may take a while to run (Cog)Shop. 
If the run is not completed then no results are returned. 
If a result is returned, but several values are `None` (namely `post run`, `messages`, `cplex`), it can be assumed that the run failed.


#### 1. Using list

Note that it may take a while to run (Cog)Shop. 
If the run is not completed then no results are returned

In [40]:
from cognite.powerops.client._generated.v1.data_classes import ShopResultList

result_list_1: ShopResultList = powerops.cogshop.list_shop_results_for_case(
    case_external_id=shop_case_1.external_id
)  # This can easily be converted to a pandas DataFrame

print(
    f"There is/are {len(result_list_1)} result(s) for the case with external id '{shop_case_1.external_id}'"
)

There is/are 1 result(s) for the case with external id 'shopcase:c2ca6f614a074a8ebbc93f18b64160f4'


#### 2. Graph QL list 

In [45]:
from cognite.powerops.client._generated.v1.data_classes import ShopResult

result_list_ql_1: list[ShopResult] = powerops.cogshop.list_shop_result_graphql(
    case_external_id=shop_case_1.external_id, limit=1
)
result_list_ql_1[0]

,value
space,power_ops_instances
external_id,shop_result__10c2856f-4faf-4611-933c-81109c5072c8
data_record,"{'version': 0, 'last_updated_time': 2024-11-11..."
node_type,None
case,"{'space': 'power_ops_instances', 'external_id'..."
objective_value,None
pre_run,POWEROPS_SHOP_pre-run-10c2856f-4faf-4611-933c-...
post_run,POWEROPS_SHOP_post-run-10c2856f-4faf-4611-933c...
messages,POWEROPS_SHOP_shop-10c2856f-4faf-4611-933c-811...
cplex_logs,POWEROPS_SHOP_cplex-10c2856f-4faf-4611-933c-81...


#### 3. Retrieve using the external ID  

In [26]:
shop_result_1: ShopResult = powerops.cogshop.retrieve_shop_result(
    result_list_1[0].external_id
)
shop_result_1

,value
space,power_ops_instances
external_id,shop_result__10c2856f-4faf-4611-933c-81109c5072c8
data_record,"{'version': 1, 'last_updated_time': 2024-11-11..."
node_type,None
case,shopcase:c2ca6f614a074a8ebbc93f18b64160f4
objective_value,"{'total': -19927822739.279236, 'load_value': 0..."
pre_run,POWEROPS_SHOP_pre-run-10c2856f-4faf-4611-933c-...
post_run,POWEROPS_SHOP_post-run-10c2856f-4faf-4611-933c...
messages,POWEROPS_SHOP_shop-10c2856f-4faf-4611-933c-811...
cplex_logs,POWEROPS_SHOP_cplex-10c2856f-4faf-4611-933c-81...


In [27]:
# Downloading one of the files to tmp directory and printing the first 10 lines
from tempfile import TemporaryDirectory

with TemporaryDirectory() as tmp_dir:
    log_file_path = str(Path(tmp_dir) / "log_file.log")
    powerops.cdf.files.download_to_path(
        external_id=shop_result_1.cplex_logs, path=log_file_path
    )
    with open(log_file_path, "r") as log_file:
        print("".join(log_file.readlines()[:10]))

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Parallel                                1
CPXPARAM_Output_CloneLog                         -1
CPXPARAM_Simplex_Limits_Iterations               10000000
CPXPARAM_MIP_Interval                            100
CPXPARAM_TimeLimit                               900
Tried aggregator 3 times.
MIP Presolve eliminated 2143 rows and 2060 columns.
MIP Presolve modified 2052 coefficients.
Aggregator did 6182 substitutions.



### Step 4 (optional): Prepare to delete created resources

In [52]:
all_external_ids_1 = [
    shop_case_1.scenario.external_id,
    shop_case_1.scenario.model.external_id,
    shop_case_1.external_id,
    shop_case_1.shop_files[0].external_id,
    shop_case_1.shop_files[1].external_id,
    shop_result_1.external_id,
]

all_external_ids_1  # Record these for deletion

['shopscenario:f6e3b243ff7a4413a1c5d32a0f88eac1',
 'shopmodel:aa8cdf7f19ef44faaf6ab3a00a7928db',
 'shopcase:c2ca6f614a074a8ebbc93f18b64160f4',
 'shopfile:a5045777f9894fdba74957c40404dc2a',
 'shopfile:31229a0ace1a4c139f8d8c7be9793fb1',
 'shop_result__10c2856f-4faf-4611-933c-81109c5072c8']

## Example 2: Reusing a ShopScenario used to set shop version

As mentioned in the intro Quickstart, the recommend way to run CogShop is by having a static model and use scenarios to vary configurations. 

However, if we know that we want to run several `ShopCases` using the same configurations, we can reuse a `ShopScenario`/`ShopModel` instead of creating several identical instances.


1. Create a `ShopScenario` and `ShopModel` that set the desired `shop_version`. Here we will create two for demo purposes
2. Write the `ShopScenario`(s) to CDF, and verify that we can retrieve them 
3. Set up a `ShopCase`, preparing shop file list with names and references as explained in Example 1 and Quickstart, and write them to CDF. 
4. Trigger shop executions of the `ShopCase`a  and list the `ShopResult` as in previous examples 

As in the previous example, we will be letting the SDK generate all external id. Again they can be 

### Step 1: Create reusable ShopScenarios with ShopScenarios

`ShopScenarios` and `ShopScenarios` created this way, will have several unused fields that are None. 
Again, this is due to the data models being intended to do more than just trigger a full case


In [11]:
# Not necessary, but we can list available files Shop Version
powerops.cogshop.list_shop_versions()

['SHOP-15.4.1.1-pyshop-python3.11-linux.zip',
 'SHOP-16.0.3-pyshop-python3.11-linux.zip',
 'SHOP-16.0.2-pyshop-python3.11-linux.zip',
 'SHOP-15.6.1.0-pyshop-python3.11-linux.zip']

In [ ]:
reusable_scenario_15_6_1_0 = powerops.cogshop.prepare_shop_scenario(
    shop_version="15.6.1.0",
    model_name="shop_model_15_5_1_0",
    scenario_name="shop_scenario_15_5_1_0",
)

reusable_scenario_16_0_3 = powerops.cogshop.prepare_shop_scenario(
    shop_version="16.0.3",
    model_name="shop_model_16_0_3",
    scenario_name="shop_scenario_16_0_3",
)

In [13]:
# Looking at the scenarios
print(f"Shop Scenario 15.6.1.0: '{reusable_scenario_15_6_1_0.external_id}'")
print(f"Shop Scenario 16.0.3:   '{reusable_scenario_16_0_3.external_id}'")

Shop Scenario 15.6.1.0: 'shopscenario:c3dd9f6ccd05401bb1d1c1cccfcbadd5'
Shop Scenario 16.0.3:   'shopscenario:d8bcee930b0d4498aabfac8a74932731'


In [14]:
# Looking at the models
reusable_scenario_16_0_3.model, reusable_scenario_15_6_1_0.model
print(f"Shop Model 15.5.1.0 '{reusable_scenario_15_6_1_0.model.external_id}'")
print(f"Shop Model 16.0.3   '{reusable_scenario_16_0_3.model.external_id}'")

Shop Model 15.5.1.0 'shopmodel:89ac1c82e9a14d72bf8be49b41b2cf20'
Shop Model 16.0.3   'shopmodel:7ad19bb0486343a7aa9fbabe1349a49a'


### Step 2: Write the scenarios to CDF

In [15]:
resources = [reusable_scenario_16_0_3, reusable_scenario_15_6_1_0]
powerops.cogshop.write_resources(resources)

ResourcesWriteResult(nodes=[<NodeApplyResult(space='power_ops_instances', external_id='shopscenario:d8bcee930b0d4498aabfac8a74932731', version=1) at 0x7f1432913bd0>, <NodeApplyResult(space='power_ops_instances', external_id='shopmodel:7ad19bb0486343a7aa9fbabe1349a49a', version=1) at 0x7f143179fdd0>, <NodeApplyResult(space='power_ops_instances', external_id='shopscenario:c3dd9f6ccd05401bb1d1c1cccfcbadd5', version=1) at 0x7f143341d0d0>, <NodeApplyResult(space='power_ops_instances', external_id='shopmodel:89ac1c82e9a14d72bf8be49b41b2cf20', version=1) at 0x7f14328de850>], edges=[], time_series=[])

### Step 3: Create new Shop Cases, referencing the newly created scenarios, and write them to CDF

We will be reusing the files from the example above

In [ ]:
#  Files are referenced using 4-tuples with the following structure:
# (file_reference, file_name, is_ascii, labels)
shop_file_list = [
    (example_case_fbu_1, example_case_fbu_1, False, ""),
    # CogShop needs to know which file contains commands in the case of multiple files
    (example_case_fbu_2, example_case_fbu_2, False, "commands"),
]

In [ ]:
shop_case_15_6_1_0 = powerops.cogshop.prepare_shop_case_with_existing_scenario(
    shop_file_list=shop_file_list,
    start_time=datetime.datetime(2023, 1, 19, 23),
    end_time=datetime.datetime(2023, 1, 29, 23),
    scenario_external_id=reusable_scenario_15_6_1_0.external_id,  # Note that we are using the external id of the scenario
)
shop_case_15_6_1_0

,value
space,power_ops_instances
external_id,shopcase:c9a0bd9b939a42b38fe0d2dec1a4ca0e
data_record,{'existing_version': None}
node_type,None
scenario,shopscenario:c3dd9f6ccd05401bb1d1c1cccfcbadd5
start_time,2023-01-19 23:00:00
end_time,2023-01-29 23:00:00
shop_files,"[{'space': 'power_ops_instances', 'external_id..."


In [ ]:
shop_case_16_0_3 = powerops.cogshop.prepare_shop_case_with_existing_scenario(
    shop_file_list=shop_file_list,
    start_time=datetime.datetime(2023, 1, 19, 23),
    end_time=datetime.datetime(2023, 1, 29, 23),
    scenario_external_id=reusable_scenario_16_0_3.external_id,  # Note that this is the external id of the scenario
)
shop_case_16_0_3

,value
space,power_ops_instances
external_id,shopcase:d8fba53cacb14c8680b680dca88735b9
data_record,{'existing_version': None}
node_type,None
scenario,shopscenario:d8bcee930b0d4498aabfac8a74932731
start_time,2023-01-19 23:00:00
end_time,2023-01-29 23:00:00
shop_files,"[{'space': 'power_ops_instances', 'external_id..."


In [ ]:
resources = [shop_case_15_6_1_0, shop_case_16_0_3]
powerops.cogshop.write_resources(resources)

ResourcesWriteResult(nodes=[<NodeApplyResult(space='power_ops_instances', external_id='shopcase:c9a0bd9b939a42b38fe0d2dec1a4ca0e', version=1) at 0x7f1431789b10>, <NodeApplyResult(space='power_ops_instances', external_id='shopfile:6c6347cd108843c7b2c9d000e8abbcd7', version=1) at 0x7f14317899d0>, <NodeApplyResult(space='power_ops_instances', external_id='shopfile:aad544cb614a476d8c8b9732458f1f88', version=1) at 0x7f143178bd10>, <NodeApplyResult(space='power_ops_instances', external_id='shopcase:d8fba53cacb14c8680b680dca88735b9', version=1) at 0x7f143178bed0>, <NodeApplyResult(space='power_ops_instances', external_id='shopfile:fdb3e6b4c7474d22a380cecde1d9ae83', version=1) at 0x7f143178bf10>, <NodeApplyResult(space='power_ops_instances', external_id='shopfile:d71c405233584d99aa5965e089f1bd79', version=1) at 0x7f143178b010>], edges=[<EdgeApplyResult(space='power_ops_instances', external_id='shopcase:c9a0bd9b939a42b38fe0d2dec1a4ca0e:shopfile:6c6347cd108843c7b2c9d000e8abbcd7', version=1) at 0

### Step 4: Trigger Shop Executions and inspect results

In [20]:
powerops.cogshop.trigger_shop_case(shop_case_15_6_1_0.external_id)
powerops.cogshop.trigger_shop_case(shop_case_16_0_3.external_id)


Note that it may take a while to run (Cog)Shop. 
If the run is not completed then no results are returned. 
If a result is returned, but several values are `None` (namely `post run`, `messages`, `cplex`), it can be assumed that the run failed.


In [ ]:
result_list_15_5_1_0: ShopResultList = powerops.cogshop.list_shop_results_for_case(
    case_external_id=shop_case_15_6_1_0.external_id
)
print(
    f"There is/are {len(result_list_15_5_1_0)} result(s) for the case '{shop_case_15_6_1_0.external_id}'"
)

There is/are 1 result(s) for the case 'shopcase:c9a0bd9b939a42b38fe0d2dec1a4ca0e'


In [46]:
shop_result_15_6_1_0: ShopResult = powerops.cogshop.retrieve_shop_result(
    result_list_15_5_1_0[0].external_id
)
shop_result_15_6_1_0

,value
space,power_ops_instances
external_id,shop_result__dad2fb07-ad9c-452a-86b5-febab8dc4966
data_record,"{'version': 1, 'last_updated_time': 2024-11-11..."
node_type,None
case,shopcase:c9a0bd9b939a42b38fe0d2dec1a4ca0e
objective_value,"{'total': -19927822739.279236, 'load_value': 0..."
pre_run,POWEROPS_SHOP_pre-run-dad2fb07-ad9c-452a-86b5-...
post_run,POWEROPS_SHOP_post-run-dad2fb07-ad9c-452a-86b5...
messages,POWEROPS_SHOP_shop-dad2fb07-ad9c-452a-86b5-feb...
cplex_logs,POWEROPS_SHOP_cplex-dad2fb07-ad9c-452a-86b5-fe...


In [ ]:
result_list_16_0_3: ShopResultList = powerops.cogshop.list_shop_result_graphql(
    case_external_id=shop_case_16_0_3.external_id, limit=1
)
print(
    f"There is/are {len(result_list_16_0_3)} result(s) for the case '{shop_case_16_0_3.external_id}'"
)

There is/are 1 result(s) for the case 'shopcase:d8fba53cacb14c8680b680dca88735b9'


In [ ]:
shop_result_16_0_3: ShopResult = powerops.cogshop.retrieve_shop_result(
    result_list_16_0_3[0].external_id
)
shop_result_16_0_3

,value
space,power_ops_instances
external_id,shop_result__188be846-2f0c-413a-b8ee-53cc52aef72b
data_record,"{'version': 1, 'last_updated_time': 2024-11-11..."
node_type,None
case,shopcase:d8fba53cacb14c8680b680dca88735b9
objective_value,"{'total': -19927822739.279236, 'load_value': 0..."
pre_run,POWEROPS_SHOP_pre-run-188be846-2f0c-413a-b8ee-...
post_run,POWEROPS_SHOP_post-run-188be846-2f0c-413a-b8ee...
messages,POWEROPS_SHOP_shop-188be846-2f0c-413a-b8ee-53c...
cplex_logs,POWEROPS_SHOP_cplex-188be846-2f0c-413a-b8ee-53...


### Step 5 (optional): Prepare to delete created resources

In [ ]:
all_external_ids_2 = [ 
    reusable_scenario_15_6_1_0.external_id,
    reusable_scenario_15_6_1_0.model.external_id,
    reusable_scenario_16_0_3.external_id,
    reusable_scenario_16_0_3.model.external_id,

    shop_case_15_6_1_0.external_id,
    shop_case_15_6_1_0.shop_files[0].external_id,
    shop_case_15_6_1_0.shop_files[1].external_id,
    
    shop_case_16_0_3.external_id,
    shop_case_16_0_3.shop_files[0].external_id,
    shop_case_16_0_3.shop_files[1].external_id,
    
    shop_result_15_6_1_0.external_id,
    shop_result_16_0_3.external_id,

]
all_external_ids_2 # Record these for deletion

['shopscenario:c3dd9f6ccd05401bb1d1c1cccfcbadd5',
 'shopmodel:89ac1c82e9a14d72bf8be49b41b2cf20',
 'shopscenario:d8bcee930b0d4498aabfac8a74932731',
 'shopmodel:7ad19bb0486343a7aa9fbabe1349a49a',
 'shopcase:c9a0bd9b939a42b38fe0d2dec1a4ca0e',
 'shopfile:6c6347cd108843c7b2c9d000e8abbcd7',
 'shopfile:aad544cb614a476d8c8b9732458f1f88',
 'shopcase:d8fba53cacb14c8680b680dca88735b9',
 'shopfile:fdb3e6b4c7474d22a380cecde1d9ae83',
 'shopfile:d71c405233584d99aa5965e089f1bd79',
 'shop_result__dad2fb07-ad9c-452a-86b5-febab8dc4966',
 'shop_result__188be846-2f0c-413a-b8ee-53cc52aef72b']

## Cleaning up the instances that were created

Instances can be deleted by using the powerops client and the `external_id`s of everything we have created 

In [55]:
all_external_ids = all_external_ids_1 + all_external_ids_2
all_external_ids

['shopscenario:f6e3b243ff7a4413a1c5d32a0f88eac1',
 'shopmodel:aa8cdf7f19ef44faaf6ab3a00a7928db',
 'shopcase:c2ca6f614a074a8ebbc93f18b64160f4',
 'shopfile:a5045777f9894fdba74957c40404dc2a',
 'shopfile:31229a0ace1a4c139f8d8c7be9793fb1',
 'shop_result__10c2856f-4faf-4611-933c-81109c5072c8',
 'shopscenario:c3dd9f6ccd05401bb1d1c1cccfcbadd5',
 'shopmodel:89ac1c82e9a14d72bf8be49b41b2cf20',
 'shopscenario:d8bcee930b0d4498aabfac8a74932731',
 'shopmodel:7ad19bb0486343a7aa9fbabe1349a49a',
 'shopcase:c9a0bd9b939a42b38fe0d2dec1a4ca0e',
 'shopfile:6c6347cd108843c7b2c9d000e8abbcd7',
 'shopfile:aad544cb614a476d8c8b9732458f1f88',
 'shopcase:d8fba53cacb14c8680b680dca88735b9',
 'shopfile:fdb3e6b4c7474d22a380cecde1d9ae83',
 'shopfile:d71c405233584d99aa5965e089f1bd79',
 'shop_result__dad2fb07-ad9c-452a-86b5-febab8dc4966',
 'shop_result__188be846-2f0c-413a-b8ee-53cc52aef72b']

In [56]:
powerops.v1.delete(external_id=all_external_ids)

InstancesDeleteResult(nodes=[NodeId(space='power_ops_instances', external_id='shopscenario:f6e3b243ff7a4413a1c5d32a0f88eac1'), NodeId(space='power_ops_instances', external_id='shopmodel:aa8cdf7f19ef44faaf6ab3a00a7928db'), NodeId(space='power_ops_instances', external_id='shopcase:c2ca6f614a074a8ebbc93f18b64160f4'), NodeId(space='power_ops_instances', external_id='shopfile:a5045777f9894fdba74957c40404dc2a'), NodeId(space='power_ops_instances', external_id='shopfile:31229a0ace1a4c139f8d8c7be9793fb1'), NodeId(space='power_ops_instances', external_id='shop_result__10c2856f-4faf-4611-933c-81109c5072c8'), NodeId(space='power_ops_instances', external_id='shopscenario:c3dd9f6ccd05401bb1d1c1cccfcbadd5'), NodeId(space='power_ops_instances', external_id='shopmodel:89ac1c82e9a14d72bf8be49b41b2cf20'), NodeId(space='power_ops_instances', external_id='shopscenario:d8bcee930b0d4498aabfac8a74932731'), NodeId(space='power_ops_instances', external_id='shopmodel:7ad19bb0486343a7aa9fbabe1349a49a'), NodeId(s